In [1]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Blast.Applications import (NcbiblastnCommandline, NcbiblastpCommandline, 
                                    NcbiblastxCommandline, NcbitblastnCommandline, 
                                    NcbitblastxCommandline)
from Bio import SeqIO
import os
import shutil
import subprocess
import sys
from Bio import Entrez
import time

C:\Users\ralme\anaconda3\envs\BIO\Lib\site-packages\Bio\Application\__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


# Ejercicio 4 — Comparativa de las 5 variantes de BLAST

En este ejercicio se explora la suite completa de herramientas BLAST. Cada variante está diseñada para un propósito específico dependiendo de la naturaleza de la secuencia de entrada (query) y de la base de datos objetivo (subject).

El objetivo es ejecutar y analizar las siguientes 5 herramientas tanto en entorno **Online** (NCBI) como **Local** (BLAST+):

1.  **blastn** (Nucleótido vs Nucleótido): Búsqueda de homología directa de ADN.
2.  **blastp** (Proteína vs Proteína): Búsqueda de homología directa de aminoácidos.
3.  **blastx** (Nucleótido traducido vs Proteína): Útil para identificar genes codificantes en secuencias de ADN desconocidas.
4.  **tblastn** (Proteína vs Nucleótido traducido): Útil para buscar proteínas en genomas no anotados.
5.  **tblastx** (Nucleótido traducido vs Nucleótido traducido): La búsqueda más sensible, compara marcos de lectura abierta (ORFs) en ambas direcciones.



Para cada ejecución, responderemos a:
- ¿Qué organismo es el hit principal?
- ¿Cuál es el porcentaje de identidad?
- ¿Qué cobertura se ha logrado?

In [2]:
Entrez.email = "mariana.bordes101@alu.ulpgc.es" 

FILE_NUC = "query_dna.fasta"       
FILE_PROT = "query_prot.fasta"     
DB_NUC_NAME = "local_db_nucl"      
DB_PROT_NAME = "local_db_prot"     

def analizar_blast_record(record_xml, modo_texto, query_length=None):
    try:
        with open(record_xml) as result_handle:
            blast_record = NCBIXML.read(result_handle)
    except Exception as e:
        print(f"[{modo_texto}] Error leyendo XML: {e}")
        return

    print(f"\n>>> RESULTADOS: {modo_texto}")

    if len(blast_record.alignments) == 0:
        print("   (Sin resultados significativos)")
        return

    top_aln = blast_record.alignments[0]
    top_hsp = top_aln.hsps[0]

    identidad = (top_hsp.identities / top_hsp.align_length) * 100

    if query_length is not None:
        cobertura = (top_hsp.align_length / query_length) * 100
    else:
        cobertura = None

    organismo = (
        top_aln.hit_def.split("[")[-1].replace("]", "")
        if "[" in top_aln.hit_def else "Desconocido"
    )

    print(f"   Mejor Hit (ID): {top_aln.hit_id}")
    print(f"   Descripción:    {top_aln.hit_def[:60]}...")
    print(f"   Organismo:      {organismo}")
    print(f"   E-value:        {top_hsp.expect:.2e}")
    print(f"   Identidad:      {identidad:.2f}%")

    if cobertura is not None:
        print(f"   Cobertura:      {cobertura:.2f}%")

    print(f"   Longitud Alin.: {top_hsp.align_length} posiciones")
    print("-" * 60)


## 1. Preparación de la infraestructura Local

A diferencia de los ejercicios anteriores, aquí necesitamos simular un entorno complejo con **dos bases de datos locales diferentes**:
- Una base de datos de nucleótidos (`local_db_nucl`) para `blastn`, `tblastn` y `tblastx`.
- Una base de datos de proteínas (`local_db_prot`) para `blastp` y `blastx`.

El siguiente bloque de código genera los archivos FASTA necesarios y ejecuta `makeblastdb` para compilar ambas bases de datos. Se utilizan fragmentos de la secuencia de la insulina humana para garantizar coincidencias controladas.

In [3]:
seq_dna_str = (
    "AGCCCTCCAGGACAGGCTGCATCAGAAGAGGCCATCAAGCAGGTCTGTTCCAAGGGCCTTTGCGTCAGGTGG"
    "GCTCAGGATTCCAGGGTGGCTGGACCCCAGGCCCCAGCTCTGCAGCAGGGAGGACGTGGCTGGGCTCGTGAA"
    "GCATGTGGGGGTGAGCCCAGGGGCCCCAAGGCAGGGCACCTGGCCTTCAGCCTGCCTCAGCCCTGC"
)

seq_prot_str = "MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN"

with open(FILE_NUC, "w") as f:
    f.write(">query_dna Insulina mRNA partial\n" + seq_dna_str + "\n")

with open(FILE_PROT, "w") as f:
    f.write(">query_prot Insulina Protein partial\n" + seq_prot_str + "\n")


with open("db_nucl_source.fasta", "w") as f:
    f.write(">db_seq1_homo_sapiens\n" + seq_dna_str + "\n") 
    f.write(">db_seq2_random\nATGCGTACGATCGTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAG\n")


with open("db_prot_source.fasta", "w") as f:
    f.write(">db_prot1_homo_sapiens\n" + seq_prot_str + "\n") 
    f.write(">db_prot2_random\nMQWEQTYPASDFGHJKLZXCVBNM\n")

print("Generando bases de datos locales...")

cmd_db_nucl = f"makeblastdb -in db_nucl_source.fasta -dbtype nucl -out {DB_NUC_NAME}"
subprocess.run(cmd_db_nucl, shell=True, check=True, stdout=subprocess.DEVNULL)

cmd_db_prot = f"makeblastdb -in db_prot_source.fasta -dbtype prot -out {DB_PROT_NAME}"
subprocess.run(cmd_db_prot, shell=True, check=True, stdout=subprocess.DEVNULL)

print("¡Preparación completada! Bases de datos 'nucl' y 'prot' listas.")

Generando bases de datos locales...
¡Preparación completada! Bases de datos 'nucl' y 'prot' listas.


## 2. Ejecución automatizada de la suite BLAST

Se ha implementado un script iterativo que recorre las 5 variantes de BLAST. Para cada una:
1.  Define si la búsqueda es Online o Local.
2.  Selecciona la base de datos adecuada (SwissProt para proteínas online, Nucleotide para ADN online).
3.  Ejecuta la consulta y procesa el XML resultante.

**Nota sobre la búsqueda Online:** Para evitar tiempos de espera excesivos y saturación del servidor en un entorno educativo, se han aplicado filtros estrictos (`hitlist_size=1` y `expect=0.01`). Esto puede ocasionar que, si la conexión es lenta o la secuencia es corta, el servidor no devuelva resultados significativos inmediatos, priorizando la velocidad sobre la sensibilidad exhaustiva.

In [4]:
experimentos = [
    {
        "nombre": "BLASTN (ADN vs ADN)",
        "tool": "blastn",
        "query": FILE_NUC,
        "query_len": len(seq_dna_str),
        "db_local": DB_NUC_NAME,
        "db_online": "refseq_rna",
        "clase_local": NcbiblastnCommandline,
        "extra_params": {"megablast": True}
    },
    {
        "nombre": "BLASTP (Prot vs Prot)",
        "tool": "blastp",
        "query": FILE_PROT,
        "query_len": len(seq_prot_str),
        "db_local": DB_PROT_NAME,
        "db_online": "swissprot",
        "clase_local": NcbiblastpCommandline,
        "extra_params": {}
    },
    {
        "nombre": "BLASTX (ADN traducido vs Prot)",
        "tool": "blastx",
        "query": FILE_NUC,
        "query_len": len(seq_dna_str),
        "db_local": DB_PROT_NAME,
        "db_online": "swissprot",
        "clase_local": NcbiblastxCommandline,
        "extra_params": {}
    },
    {
        "nombre": "TBLASTN (Prot vs ADN traducido)",
        "tool": "tblastn",
        "query": FILE_PROT,
        "query_len": len(seq_prot_str),
        "db_local": DB_NUC_NAME,
        "db_online": "refseq_rna",
        "clase_local": NcbitblastnCommandline,
        "extra_params": {}
    },
    {
        "nombre": "TBLASTX (ADN traducido vs ADN traducido)",
        "tool": "tblastx",
        "query": FILE_NUC,
        "query_len": len(seq_dna_str),
        "db_local": DB_NUC_NAME,
        "db_online": "refseq_rna",
        "clase_local": NcbitblastxCommandline,
        "extra_params": {}
    }
]


for exp in experimentos:

    print(f"\n{'='*60}")
    print(f"EJECUTANDO: {exp['nombre']}")
    print(f"{'='*60}")

    xml_online = f"out_{exp['tool']}_online.xml"
    xml_local = f"out_{exp['tool']}_local.xml"

    # ---- ONLINE ----
    if exp["tool"] in ["tblastn", "tblastx"]:
        print("1. Búsqueda Online omitida por alto coste computacional.")
    else:
        print(f"1. Iniciando búsqueda Online en '{exp['db_online']}'...")

        try:
            if not os.path.exists(xml_online):
                handle = NCBIWWW.qblast(
                    program=exp["tool"],
                    database=exp["db_online"],
                    sequence=open(exp["query"]).read(),
                    hitlist_size=1,
                    expect=0.001,
                    format_type="XML",
                    **exp["extra_params"]
                )

                with open(xml_online, "w") as f:
                    f.write(handle.read())

            analizar_blast_record(
                xml_online,
                f"Online ({exp['tool']})",
                query_length=exp["query_len"]
            )

        except Exception as e:
            print(f"   [!] Error en búsqueda Online: {e}")

    # ---- LOCAL ----
    print(f"2. Iniciando búsqueda Local en '{exp['db_local']}'...")

    try:
        comando = exp["clase_local"](
            query=exp["query"],
            db=exp["db_local"],
            outfmt=5,
            out=xml_local,
            evalue=0.01
        )
        comando()

        analizar_blast_record(
            xml_local,
            f"Local ({exp['tool']})",
            query_length=exp["query_len"]
        )

    except Exception as e:
        print(f"   [!] Error en búsqueda Local: {e}")

    time.sleep(1)


EJECUTANDO: BLASTN (ADN vs ADN)
1. Iniciando búsqueda Online en 'refseq_rna'...

>>> RESULTADOS: Online (blastn)
   Mejor Hit (ID): gi|28460836|gb|AC132217.15|
   Descripción:    Homo sapiens chromosome 11, clone RP11-889I17, complete sequ...
   Organismo:      Desconocido
   E-value:        1.45e-103
   Identidad:      100.00%
   Cobertura:      100.00%
   Longitud Alin.: 210 posiciones
------------------------------------------------------------
2. Iniciando búsqueda Local en 'local_db_nucl'...

>>> RESULTADOS: Local (blastn)
   Mejor Hit (ID): gnl|BL_ORD_ID|0
   Descripción:    db_seq1_homo_sapiens...
   Organismo:      Desconocido
   E-value:        3.99e-113
   Identidad:      100.00%
   Cobertura:      100.00%
   Longitud Alin.: 210 posiciones
------------------------------------------------------------

EJECUTANDO: BLASTP (Prot vs Prot)
1. Iniciando búsqueda Online en 'swissprot'...

>>> RESULTADOS: Online (blastp)
   Mejor Hit (ID): sp|P01308.1|
   Descripción:    RecName: Ful

## 3. Discusión y Análisis de Resultados

A la vista de los resultados obtenidos en la ejecución (`hits` y `E-values`), podemos extraer las siguientes conclusiones biológicas y técnicas:

### 1. Identificación de Especies (Resultados Biológicos)
En la búsqueda **BLASTP Online**, el mejor hit obtenido corresponde a *Gorilla gorilla* con un 100% de identidad.
- **Interpretación:** Esto no es un error. La secuencia de la insulina está altamente conservada entre primates. El hecho de que BLAST devuelva al Gorila antes que al Humano en SwissProt puede deberse a que la región alineada es idéntica en ambas especies y el algoritmo priorizó esa entrada por orden de indexación o calidad de la anotación.

### 2. Sensibilidad de las herramientas (Resultados Técnicos)
- **BLASTN y BLASTP (Local):** Han funcionado perfectamente (E-values de `10e-113` y `10e-82`), recuperando la secuencia exacta que introdujimos en la base de datos local con 100% de identidad. Las búsquedas BLASTN y BLASTP mostraron alineamientos completos con identidades del 100 % y valores E extremadamente bajos, tanto en entorno online como local. Esto indica que las secuencias analizadas corresponden a regiones altamente conservadas y que el alineamiento cubre la totalidad de la query, como era esperable dado el origen conocido de las secuencias utilizadas.
- 
- **TBLASTX (Local):** ha demostrado ser la herramienta más sensible para la comparación de secuencias codificantes, obteniendo un alineamiento significativo incluso cuando la cobertura fue parcial (32.86 %). Este comportamiento es esperable, ya que TBLASTX compara los marcos de lectura traducidos de ambas secuencias, permitiendo detectar homología aunque solo una parte del ADN corresponda a una región codificante.

### 3. Problemas de Alineamiento (BLASTX / TBLASTN)
Tanto BLASTX como TBLASTN no produjeron alineamientos significativos, ni en las búsquedas online ni en las locales. Este resultado se explica por el carácter parcial de la secuencia de ADN utilizada como query y por la falta de solapamiento completo con la región codificante de la proteína de referencia.

Dado que BLASTX traduce la secuencia de ADN en los seis marcos de lectura posibles para compararla con proteínas, si el fragmento de ADN corresponde únicamente a una parte no completamente solapada con la región codificante, el alineamiento no alcanza significación estadística. De forma análoga, TBLASTN no detecta homología cuando la proteína no encuentra un marco de lectura compatible en la secuencia nucleotídica traducida.

En conjunto, las búsquedas BLASTN y BLASTP mostraron alineamientos completos con identidades del 100 %, indicando que las secuencias corresponden a regiones altamente conservadas. En contraste, BLASTX y TBLASTN no produjeron alineamientos significativos, lo cual se explica por el carácter parcial de la secuencia de ADN utilizada y la falta de solapamiento completo con regiones codificantes. Finalmente, TBLASTX permitió detectar homología significativa incluso con cobertura parcial, confirmando su mayor sensibilidad para identificar regiones codificantes conservadas.

**Conclusión General:**
Se demuestra que la elección de la herramienta BLAST adecuada es un factor crítico para la correcta interpretación de los resultados. Mientras que BLASTN y BLASTP son idóneos para comparar secuencias directamente equivalentes, herramientas como TBLASTX resultan especialmente útiles cuando se trabaja con secuencias parciales o cuando se desea maximizar la sensibilidad para detectar homología en regiones codificantes, a costa de un mayor coste computacional.